# Data Quality Review & Light Cleaning: `NSMES1988 - NSMES1988.csv`

This notebook guides a **data quality review** for the CSV file
`NSMES1988 - NSMES1988.csv`. We will:

- Load the CSV and configure display options for full visibility.
- Inspect shape, columns, data types, and summary statistics.
- Check for **missing values** and basic anomalies.
- Spot-check **row 287** (0-based index), where we noticed an **age value of 8**.
- Apply a **safe age-scaling rule** that *only* multiplies decimal-like ages
  under 10 (e.g., `2.8 → 28`) while **leaving true ages like 8 untouched**.
- Scale `income` by ×1000 (keeping decimals visible).
- Save a **cleaned** version to a new CSV.

> ℹ️ The key caution: some rows have ages recorded as decimals (e.g., `2.8` meaning `28`).
> We must avoid turning a real `8`-year-old into `80`. We'll handle that safely.


## 1) Setup & Imports

Configure pandas display to avoid truncated outputs while inspecting data.


In [ ]:
import numpy as np
import pandas as pd

# Display options for thorough inspection (use carefully for very large CSVs)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)


## 2) Load Data

We load the CSV and drop the default index column if it exists (often appears as `Unnamed: 0`).

In [ ]:
# Path to your data file
csv_path = "NSMES1988 - NSMES1988.csv"

# Load and drop any default index columns like 'Unnamed: 0'
df_raw = pd.read_csv(csv_path)
df = df_raw.loc[:, ~df_raw.columns.astype(str).str.match(r"^Unnamed")].copy()

print("Loaded shape:", df.shape)
df.head(10)  # Show a quick peek


## 3) Overview: Columns, Types, Summary

Check column names, data types, and get a statistical snapshot.


In [ ]:
print("Columns:", list(df.columns))
print("\nData types:")
print(df.dtypes)

print("\n.info():")
df.info()

print("\n.describe() numeric:")
display(df.describe())

print("\n.describe(include='object') categorical:")
display(df.describe(include='object'))


## 4) Missing Values

Count how many missing values appear per column, and which rows contain *any* missing values.


In [ ]:
print("Missing values per column:")
print(df.isnull().sum())

print("\nTotal missing values:", int(df.isnull().sum().sum()))

print("\nRows with any missing values (showing first 10):")
display(df[df.isnull().any(axis=1)].head(10))


## 5) Spot Check: Row 287 (0-based index)

We noticed that **row 287** has an `age` value of **8**. That likely represents a genuine 8-year-old,
so we **must not** multiply it by 10 (to avoid turning 8 into 80). We’ll confirm below.


In [ ]:
row_idx = 287  # 0-based index
print(f"Row {row_idx} preview:")
display(df.iloc[[row_idx]])

age_val = df.loc[df.index[row_idx], 'age'] if 'age' in df.columns else None
print("Age at row 287:", age_val)


## 6) Data Quality Rule for `age`

**Goal:** Some ages appear to be stored as decimals under 10 (e.g., `2.8` meaning `28`).  
We want to **scale only those decimal-like ages under 10**, while leaving true integer ages (like `8`) untouched.

**Rule:** For any `age` value `< 10` where the value **is not an integer** (i.e., has a fractional part), multiply by 10.


In [ ]:
# Identify candidate rows where 'age' is < 10 and has a fractional part
def has_fraction(x):
    try:
        return pd.notnull(x) and float(x) < 10 and not float(x).is_integer()
    except Exception:
        return False

candidates = df['age'].apply(has_fraction)

print("Count of decimal-like ages < 10 (to be scaled):", int(candidates.sum()))
display(df.loc[candidates, ['age']].head(10))

# Also show small ages that are integers (e.g., 8) that should NOT be scaled
small_integers = df['age'].apply(lambda x: pd.notnull(x) and float(x) < 10 and float(x).is_integer())
print("\nCount of small integer ages < 10 (will remain as-is):", int(small_integers.sum()))
display(df.loc[small_integers, ['age']].head(10))


## 7) Apply Transformations

- **Age:** Scale only decimal-like ages under 10 by ×10 (leave integer ages under 10 unchanged).
- **Income:** Multiply by ×1000 and keep decimals visible.


In [ ]:
df_clean = df.copy()

# Apply safe age scaling
def scale_age(x):
    try:
        fx = float(x)
        if fx < 10 and not fx.is_integer():
            return fx * 10
        return fx
    except Exception:
        return x

if 'age' in df_clean.columns:
    df_clean['age'] = df_clean['age'].apply(scale_age)

# Scale income by ×1000 without dropping decimals
if 'income' in df_clean.columns:
    df_clean['income'] = pd.to_numeric(df_clean['income'], errors='coerce') * 1000

# Quick verification around row 287
print("Row 287 after transformation:")
display(df_clean.iloc[[287]])


## 8) Verify Changes

Show a few rows where age was adjusted, and confirm that an actual `8`-year-old (row 287) is **not** scaled to `80`.


In [ ]:
# Rows where age changed
if 'age' in df.columns:
    changed = df_clean['age'] != df['age']
    print("Number of rows where 'age' changed:", int(changed.sum()))
    display(pd.concat([df.loc[changed, ['age']].rename(columns={'age': 'age_before'}),
                       df_clean.loc[changed, ['age']].rename(columns={'age': 'age_after'})],
                      axis=1).head(10))
else:
    print("'age' column not found; skipping comparison.")


## 9) Save Cleaned Data

Write out a cleaned CSV for downstream analysis.


In [ ]:
out_path = "/mnt/data/NSMES1988_clean.csv"
df_clean.to_csv(out_path, index=False)
out_path


## 10) Summary

- Loaded and inspected `NSMES1988 - NSMES1988.csv`.
- Found potential mixed-format **ages**: decimals under 10 that likely represent tens (e.g., `2.8 → 28`).
- **Row 287** had `age = 8`, which we treated as a legitimate 8-year-old and **did not scale**.
- Scaled only decimal-like ages `< 10` by ×10.
- Scaled `income` by ×1000 (kept decimals).
- Saved cleaned output to `/mnt/data/NSMES1988_clean.csv`.
